In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1hzmxNBrY7-9mv5EpqAvhVUiJahfrcYUN'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('epi_r.csv')

In [ ]:
import re
import time
import pickle
import pprint
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from getpass import getpass
from itertools import product
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, mean_squared_error

In [ ]:
api_key = getpass()

··········


In [ ]:
pp = pprint.PrettyPrinter(compact=True)

#Preprocessing & Feature selection

In [ ]:
df = pd.read_csv('epi_r.csv')

In [ ]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,alabama,alaska,alcoholic,almond,amaretto,anchovy,anise,anniversary,anthony bourdain,aperitif,appetizer,apple,apple juice,apricot,arizona,artichoke,arugula,asian pear,asparagus,aspen,atlanta,australia,avocado,back to school,backyard bbq,bacon,bake,banana,...,vegan,vegetable,vegetarian,venison,vermont,vermouth,vinegar,virginia,vodka,waffle,walnut,wasabi,washington,"washington, d.c.",watercress,watermelon,wedding,weelicious,west virginia,westwood,wheat/gluten-free,whiskey,white wine,whole wheat,wild rice,windsor,wine,winter,wisconsin,wok,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pp.pprint(list(df.columns))

['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '#cakeweek',
 '#wasteless', '22-minute meals', '3-ingredient recipes',
 '30 days of groceries', 'advance prep required', 'alabama', 'alaska',
 'alcoholic', 'almond', 'amaretto', 'anchovy', 'anise', 'anniversary',
 'anthony bourdain', 'aperitif', 'appetizer', 'apple', 'apple juice', 'apricot',
 'arizona', 'artichoke', 'arugula', 'asian pear', 'asparagus', 'aspen',
 'atlanta', 'australia', 'avocado', 'back to school', 'backyard bbq', 'bacon',
 'bake', 'banana', 'barley', 'basil', 'bass', 'bastille day', 'bean', 'beef',
 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper',
 'berry', 'beverly hills', 'birthday', 'biscuit', 'bitters', 'blackberry',
 'blender', 'blue cheese', 'blueberry', 'boil', 'bok choy', 'bon appétit',
 'bon app��tit', 'boston', 'bourbon', 'braise', 'bran', 'brandy', 'bread',
 'breadcrumbs', 'breakfast', 'brie', 'brine', 'brisket', 'broccoli',
 'broccoli rabe', 'broil', 'brooklyn', 'brown 

Объединяем столбцы с одинаковыми ингредиентами.

In [ ]:
df['cookie'] = np.maximum(df['cookie'], df['cookies'])
df['créme de cacao'] = np.maximum(df['créme de cacao'], df['cr��me de cacao'])

Оставляем только признаки, обозначающие ингредиенты (только те, которые есть в базе данных FNDDS).

In [ ]:
df = df.filter(items=['rating', 'amaretto', 'anchovy', 'apple', 'apple juice',
                      'asparagus', 'bacon', 'barley', 'basil', 'bass', 'bean',
                      'beef rib', 'beer', 'beet', 'blackberry', 'blue cheese',
                      'blueberry', 'bok choy', 'brandy', 'bread', 'brie',
                      'broccoli', 'brown rice', 'brussel sprout', 'burrito',
                      'butter', 'butterscotch/caramel', 'cabbage', 'candy',
                      'carrot', 'casserole/gratin', 'cauliflower', 'caviar',
                      'celery', 'champagne', 'chard', 'chartreuse', 'cheddar',
                      'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea',
                      'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro',
                      'cinnamon', 'cobbler/crumble', 'coconut', 'coffee',
                      'cognac/armagnac', 'collard greens', 'cookie',
                      'coriander', 'cottage cheese', 'couscous', 'crab',
                      'cranberry', 'cranberry sauce', 'cream cheese',
                      'créme de cacao', 'cucumber', 'currant', 'custard',
                      'eau de vie', 'endive', 'escarole', 'fennel', 'feta',
                      'fig', 'fish', 'fontina', 'gin', 'ginger', 'goat cheese',
                      'gouda', 'grains', 'grapefruit', 'green bean',
                      'green onion/scallion', 'ground beef', 'ground lamb',
                      'halibut', 'ham', 'hamburger', 'hazelnut', 'honey',
                      'honeydew', 'horseradish', 'hot pepper', 'ice cream',
                      'iced coffee', 'iced tea', 'jam or jelly',
                      'jerusalem artichoke', 'kale', 'kiwi', 'kumquat', 'lamb',
                      'lamb chop', 'lasagna', 'leek', 'lemon', 'lemon juice',
                      'lentil', 'lettuce', 'lima bean', 'lime', 'lime juice',
                      'liqueur', 'lobster', 'lychee', 'macadamia nut',
                      'macaroni and cheese', 'maple syrup', 'margarita',
                      'marinade', 'marsala', 'marshmallow', 'martini',
                      'mayonnaise', 'meat', 'meatball', 'meatloaf', 'melon',
                      'milk/cream', 'molasses', 'monterey jack', 'mozzarella',
                      'muffin', 'mushroom', 'mussel', 'mustard',
                      'mustard greens', 'nectarine', 'noodle', 'oat', 'oatmeal',
                      'octopus', 'okra', 'olive', 'omelet', 'onion', 'orange',
                      'orange juice', 'oyster', 'pancake', 'parmesan',
                      'parsley', 'parsnip', 'pasta', 'pea', 'peach', 'peanut',
                      'peanut butter', 'pear', 'pecan', 'persimmon', 'pie',
                      'pine nut', 'pineapple', 'pistachio', 'pizza', 'plantain',
                      'plum', 'pomegranate', 'pomegranate juice', 'poppy',
                      'pork', 'pork chop', 'pork tenderloin', 'potato',
                      'potato salad', 'poultry', 'poultry sausage',
                      'prosciutto', 'prune', 'pumpkin', 'punch', 'quail',
                      'quiche', 'quinoa', 'rabbit', 'radicchio', 'radish',
                      'raisin', 'red wine', 'rhubarb', 'rice', 'ricotta', 'rum',
                      'rutabaga', 'rye', 'sake', 'salad dressing', 'salmon',
                      'salsa', 'sandwich', 'sangria', 'sardine', 'sauce',
                      'sausage', 'scotch', 'seafood', 'semolina', 'sesame',
                      'sesame oil', 'shellfish', 'shrimp', 'smoothie', 'sorbet',
                      'soup/stew', 'sour cream', 'soy sauce', 'sparkling wine',
                      'spinach', 'squid', 'steak', 'stew', 'strawberry',
                      'sugar snap pea', 'sweet potato/yam', 'swiss cheese',
                      'swordfish', 'taco', 'tamarind', 'tangerine', 'tapioca',
                      'tart', 'tea', 'tequila', 'tilapia', 'tofu', 'tomato',
                      'tortillas', 'tree nut', 'trout', 'tuna', 'turkey',
                      'turnip', 'vanilla', 'venison', 'vermouth', 'vinegar',
                      'vodka', 'waffle', 'walnut', 'wasabi', 'watercress',
                      'watermelon', 'whiskey', 'white wine', 'whole wheat',
                      'wild rice', 'wine', 'yellow squash', 'yogurt', 'yuca',
                      'zucchini'], axis=1)

У признаков-ингредиентов с двумя названиями оставляем одно.

In [ ]:
df = df.rename(columns=lambda x: re.sub(r'/[\w ]*', '', x))
df = df.rename(columns={'jam or jelly': 'jam'})

In [ ]:
df.head()

,rating,amaretto,anchovy,apple,apple juice,asparagus,bacon,barley,basil,bass,bean,beef rib,beer,beet,blackberry,blue cheese,blueberry,bok choy,brandy,bread,brie,broccoli,brown rice,brussel sprout,burrito,butter,butterscotch,cabbage,candy,carrot,casserole,cauliflower,caviar,celery,champagne,chard,chartreuse,cheddar,cheese,cherry,...,strawberry,sugar snap pea,sweet potato,swiss cheese,swordfish,taco,tamarind,tangerine,tapioca,tart,tea,tequila,tilapia,tofu,tomato,tortillas,tree nut,trout,tuna,turkey,turnip,vanilla,venison,vermouth,vinegar,vodka,waffle,walnut,wasabi,watercress,watermelon,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,yuca,zucchini
0,2.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Осталось в 2.6 раза меньше столбцов!

In [ ]:
pp.pprint(list(df.columns))

['rating', 'amaretto', 'anchovy', 'apple', 'apple juice', 'asparagus', 'bacon',
 'barley', 'basil', 'bass', 'bean', 'beef rib', 'beer', 'beet', 'blackberry',
 'blue cheese', 'blueberry', 'bok choy', 'brandy', 'bread', 'brie', 'broccoli',
 'brown rice', 'brussel sprout', 'burrito', 'butter', 'butterscotch', 'cabbage',
 'candy', 'carrot', 'casserole', 'cauliflower', 'caviar', 'celery', 'champagne',
 'chard', 'chartreuse', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken',
 'chickpea', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro',
 'cinnamon', 'cobbler', 'coconut', 'coffee', 'cognac', 'collard greens',
 'cookie', 'coriander', 'cottage cheese', 'couscous', 'crab', 'cranberry',
 'cranberry sauce', 'cream cheese', 'créme de cacao', 'cucumber', 'currant',
 'custard', 'eau de vie', 'endive', 'escarole', 'fennel', 'feta', 'fig', 'fish',
 'fontina', 'gin', 'ginger', 'goat cheese', 'gouda', 'grains', 'grapefruit',
 'green bean', 'green onion', 'ground beef', 'ground lamb', 'halibu

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('rating', axis=1), df['rating'], random_state=42)

#Regression
Посчитаем RMSE наивной регрессии.

In [ ]:
y_pred_naive = np.full(y_test.shape, np.mean(y_train))

In [ ]:
y_pred_naive

array([3.70952191, 3.70952191, 3.70952191, ..., 3.70952191, 3.70952191,
       3.70952191])

In [ ]:
mean_squared_error(y_test, y_pred_naive, squared=False)

1.3243852365612345

In [ ]:
linear_reg = LinearRegression()

param_grid = [{'fit_intercept': [True], 'normalize': [True, False]},
              {'fit_intercept': [False]}]

cv_linear_reg = GridSearchCV(linear_reg, param_grid,
                             scoring='neg_root_mean_squared_error', n_jobs=-1)

In [ ]:
cv_linear_reg.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'fit_intercept': [True], 'normalize': [True, False]},
                         {'fit_intercept': [False]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_root_mean_squared_error', verbose=0)

In [ ]:
cv_linear_reg.best_params_

{'fit_intercept': True, 'normalize': False}

In [ ]:
linear_reg = LinearRegression()

In [ ]:
linear_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred = linear_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

2749696142.4940658

RMSE линейной регрессии очень высокая.

In [ ]:
tree_reg = DecisionTreeRegressor()

param_grid = {'criterion': ['mse', 'friedman_mse'],
              'splitter': ['best', 'random'],
              'max_depth': np.arange(1, 50),
              'random_state': [42]}

cv_tree_reg = GridSearchCV(tree_reg, param_grid,
                           scoring='neg_root_mean_squared_error', n_jobs=-1,
                           verbose=2)

In [ ]:
cv_tree_reg.fit(X_train, y_train)

Fitting 5 folds for each of 196 candidates, totalling 980 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 296 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 544 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 920 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 980 out of 980 | elapsed:  3.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['mse', 'friedman_mse'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12

In [ ]:
cv_tree_reg.best_params_

{'criterion': 'mse', 'max_depth': 7, 'random_state': 42, 'splitter': 'random'}

In [ ]:
tree_reg = DecisionTreeRegressor(splitter='random', max_depth=7,
                                 random_state=42)

In [ ]:
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=7,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='random')

In [ ]:
y_pred = tree_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

1.3022615221857625

RMSE решающего дерева лучше, чем у наивной регрессии.

In [ ]:
forest_reg = RandomForestRegressor(n_jobs=-1)
param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['mse', 'friedman_mse'],
              'max_depth': np.arange(1, 50),
              'random_state': [42]}

cv_forest_reg = GridSearchCV(forest_reg, param_grid,
                             scoring='neg_root_mean_squared_error', n_jobs=-1,
                             verbose=2)

In [ ]:
cv_forest_reg.fit(X_train, y_train)

Fitting 5 folds for each of 392 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 41.6min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 90.1min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 113.5min
[Parallel(n_jobs=-1)]: Done 1960 out of 1960 | elapsed: 183.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             oob_score=False, ran...
             param_grid={'criterion': ['mse', 'friedman_mse'],
                         'max_depth': array([ 1,  2,  3,  

In [ ]:
cv_forest_reg.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 12,
 'n_estimators': 100,
 'random_state': 42}

In [ ]:
forest_reg = RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                   n_jobs=-1, random_state=42)

In [ ]:
forest_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='friedman_mse',
                      max_depth=12, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
y_pred = forest_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

1.2887423021027884

RMSE случайного дерева лучше, чем у решающего дерева.
Для ансамблевых методов будем использовать решающее дерево и случайный лес с подобранными параметрами. Попробуем VotingRegressor с разными весами.

In [ ]:
tree_reg = DecisionTreeRegressor(splitter='random', max_depth=7,
                                 random_state=42)
forest_reg = RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                   n_jobs=-1, random_state=42)

voting_reg = VotingRegressor(
    [('tree', tree_reg), ('forest', forest_reg)], n_jobs=-1)

In [ ]:
param_grid = {'weights': list(product(range(1, 6), repeat=2))}

cv_voting_reg = GridSearchCV(voting_reg, param_grid,
                             scoring='neg_root_mean_squared_error', n_jobs=-1,
                             verbose=2)

In [ ]:
cv_voting_reg.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed: 15.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=VotingRegressor(estimators=[('tree',
                                                    DecisionTreeRegressor(ccp_alpha=0.0,
                                                                          criterion='mse',
                                                                          max_depth=7,
                                                                          max_features=None,
                                                                          max_leaf_nodes=None,
                                                                          min_impurity_decrease=0.0,
                                                                          min_impurity_split=None,
                                                                          min_samples_leaf=1,
                                                                          min_samples_split=2,
                                                         

In [ ]:
cv_voting_reg.best_params_

{'weights': (1, 5)}

У решающего дерева наименьший вес, у случайного леса наибольший.

In [ ]:
tree_reg = DecisionTreeRegressor(splitter='random', max_depth=7,
                                 random_state=42)
forest_reg = RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                   n_jobs=-1, random_state=42)

voting_reg = VotingRegressor(
    [('tree', tree_reg), ('forest', forest_reg)], weights=(1, 5), n_jobs=-1)

In [ ]:
voting_reg.fit(X_train, y_train)

VotingRegressor(estimators=[('tree',
                             DecisionTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse', max_depth=7,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort='deprecated',
                                                   random_state=42,
                                                   splitter='random')),
                            ('forest',
                             RandomForestReg

In [ ]:
y_pred = voting_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

1.289498617733663

RMSE лучше, чем у одного решающего дерева, но хуже, чем у одного случайного леса. Попробуем вообще не использовать решающее дерево – возьмём BaggingRegressor на случайном лесе с разными значениями n_estimators.

In [ ]:
bagging_reg = BaggingRegressor(
    base_estimator=RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                         random_state=42), n_jobs=-1)

In [ ]:
param_grid = {'n_estimators': [5, 10, 15],
              'random_state': [42]}

cv_bagging_reg = GridSearchCV(bagging_reg, param_grid,
                             scoring='neg_root_mean_squared_error', n_jobs=-1,
                             verbose=2)

In [ ]:
cv_bagging_reg.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 12.6min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingRegressor(base_estimator=RandomForestRegressor(bootstrap=True,
                                                                             ccp_alpha=0.0,
                                                                             criterion='friedman_mse',
                                                                             max_depth=12,
                                                                             max_features='auto',
                                                                             max_leaf_nodes=None,
                                                                             max_samples=None,
                                                                             min_impurity_decrease=0.0,
                                                                             min_impurity_split=None,
                                                                             min_samp

In [ ]:
cv_bagging_reg.best_params_

{'n_estimators': 15, 'random_state': 42}

In [ ]:
bagging_reg = BaggingRegressor(
    base_estimator=RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                         random_state=42), n_estimators=15,
                               n_jobs=-1)

In [ ]:
bagging_reg.fit(X_train, y_train)

BaggingRegressor(base_estimator=RandomForestRegressor(bootstrap=True,
                                                      ccp_alpha=0.0,
                                                      criterion='friedman_mse',
                                                      max_depth=12,
                                                      max_features='auto',
                                                      max_leaf_nodes=None,
                                                      max_samples=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      n_estimators=100,
                          

In [ ]:
y_pred = bagging_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

1.2879127452545764

RMSE чуть-чуть лучше, чем у одного случайного леса. Попробуем StackingRegressor на решающем дереве и случайном лесе с разными значениями cv и passthrough.

In [ ]:
tree_reg = DecisionTreeRegressor(splitter='random', max_depth=7,
                                 random_state=42)
forest_reg = RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                   n_jobs=-1, random_state=42)

stacking_reg = StackingRegressor(
    [('tree', tree_reg), ('forest', forest_reg)], n_jobs=-1)

In [ ]:
param_grid = {'cv': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'passthrough': [True, False]}

cv_stacking_reg = GridSearchCV(stacking_reg, param_grid,
                             scoring='neg_root_mean_squared_error', n_jobs=-1,
                             verbose=2)

In [ ]:
cv_stacking_reg.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 67.6min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=StackingRegressor(cv=None,
                                         estimators=[('tree',
                                                      DecisionTreeRegressor(ccp_alpha=0.0,
                                                                            criterion='mse',
                                                                            max_depth=7,
                                                                            max_features=None,
                                                                            max_leaf_nodes=None,
                                                                            min_impurity_decrease=0.0,
                                                                            min_impurity_split=None,
                                                                            min_samples_leaf=1,
                                                                            min_samp

In [ ]:
cv_stacking_reg.best_params_

{'cv': 3, 'passthrough': True}

In [ ]:
tree_reg = DecisionTreeRegressor(splitter='random', max_depth=7,
                                 random_state=42)
forest_reg = RandomForestRegressor(criterion='friedman_mse', max_depth=12,
                                   n_jobs=-1, random_state=42)

stacking_reg = StackingRegressor(
    [('tree', tree_reg), ('forest', forest_reg)], cv=3, n_jobs=-1,
    passthrough=True)

In [ ]:
stacking_reg.fit(X_train, y_train)

StackingRegressor(cv=3,
                  estimators=[('tree',
                               DecisionTreeRegressor(ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=7,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
                                                     random_state=42,
                                                     s

In [ ]:
y_pred = stacking_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

1.2745584197651227

StackingRegressor с подобранными параметрами даёт самую низкую RMSE на тестовой выборке!
#6-class classification

In [ ]:
y_train, y_test = np.rint(y_train).astype(int), np.rint(y_test).astype(int)

Посчитаем accuracy наивного классификатора.

In [ ]:
y_pred_naive = np.full(y_test.shape, np.bincount(y_train).argmax())

In [ ]:
y_pred_naive

array([4, 4, 4, ..., 4, 4, 4])

In [ ]:
accuracy_score(y_test, y_pred_naive)

0.661280670257331

In [ ]:
logreg_clf = LogisticRegression()

param_grid = {'penalty': ['l2', 'none'],
              'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'fit_intercept': [True, False],
              'class_weight': ['balanced', None],
              'random_state': [42],
              'solver': ['newton-sg', 'lbfgs', 'sag', 'saga']}

cv_logreg_clf = GridSearchCV(logreg_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_logreg_clf.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 369 tasks      | elapsed: 18.0min
[Parallel(n_jobs=-1)]: Done 652 tasks      | elapsed: 34.4min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 55.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', None],
                         'fit_intercept': [True, False],
                         'penalty': ['l2', 'none'], 'random_state': [42],
                         'solver': ['newton-sg', 'lbfgs', 'sag', 'saga']},
             pre_dispatch=

In [ ]:
cv_logreg_clf.best_params_

{'C': 0.1,
 'class_weight': None,
 'fit_intercept': True,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs'}

In [ ]:
logreg_clf = LogisticRegression(C=0.1, random_state=42, n_jobs=-1)

In [ ]:
logreg_clf.fit(X_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=42,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
y_pred = logreg_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.6668661480151605

Accuracy логистической регрессии чуть-чуть лучше наивного классификатора. <br>
SVM не подходит для больших датасетов, поэтому её здесь использовать не будем.

In [ ]:
tree_clf = DecisionTreeClassifier()

param_grid = {'criterion': ['gini','entropy'],
              'splitter': ['best', 'random'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [42]}

cv_tree_clf = GridSearchCV(tree_clf, param_grid, scoring='accuracy', n_jobs=-1,
                           verbose=2)

In [ ]:
cv_tree_clf.fit(X_train, y_train)

Fitting 5 folds for each of 392 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 294 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done 503 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 928 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1399 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1944 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 1960 out of 1960 | elapsed:  7.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid=...
             param_grid={'class_weight': ['balanced', None],
                         'criterion':

In [ ]:
cv_tree_clf.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'random_state': 42,
 'splitter': 'best'}

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)

In [ ]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [ ]:
y_pred = tree_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.6648713345302214

Решающее дерево хуже логистической регрессии и даже хуже наивного классификатора.

In [ ]:
forest_clf = RandomForestClassifier(n_jobs=-1)

param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'random_state': [42]}

cv_forest_clf = GridSearchCV(forest_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_forest_clf.fit(X_train, y_train)

Fitting 5 folds for each of 392 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   37.4s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 1960 out of 1960 | elapsed: 23.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1...
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
          

Алгоритмы классификации обучаются значительно быстрее регрессии.

In [ ]:
cv_forest_clf.best_params_

{'criterion': 'gini', 'max_depth': 44, 'n_estimators': 100, 'random_state': 42}

In [ ]:
forest_clf = RandomForestClassifier(max_depth=44, n_jobs=-1, random_state=42)

In [ ]:
forest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=44, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = forest_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.6714542190305206

Уже значительно лучше!<br>
Для ансамблевых методов будем использовать все три классификатора.

In [ ]:
logreg_clf = LogisticRegression(C=0.1, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)
forest_clf = RandomForestClassifier(max_depth=44, n_jobs=-1, random_state=42)

voting_clf = VotingClassifier([('logreg', logreg_clf),
                               ('tree', tree_clf),
                               ('forest', forest_clf)], n_jobs=-1)

In [ ]:
param_grid = {'voting': ['hard', 'soft'],
              'weights': list(product(range(1, 6), repeat=3))}

cv_voting_clf = GridSearchCV(voting_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_voting_clf.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 37.2min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 58.1min
[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 71.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=VotingClassifier(estimators=[('logreg',
                                                     LogisticRegression(C=0.1,
                                                                        class_weight=None,
                                                                        dual=False,
                                                                        fit_intercept=True,
                                                                        intercept_scaling=1,
                                                                        l1_ratio=None,
                                                                        max_iter=100,
                                                                        multi_class='auto',
                                                                        n_jobs=-1,
                                                                        penalty='l2',
                         

In [ ]:
cv_voting_clf.best_params_

{'voting': 'hard', 'weights': (1, 1, 3)}

Снова вес случайного леса выше, чем вес остальных моделей.

In [ ]:
logreg_clf = LogisticRegression(C=0.1, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)
forest_clf = RandomForestClassifier(max_depth=44, n_jobs=-1, random_state=42)

voting_clf = VotingClassifier([('logreg', logreg_clf),
                               ('tree', tree_clf),
                               ('forest', forest_clf)], weights=(1, 1, 3),
                              n_jobs=-1)

In [ ]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('logreg',
                              LogisticRegression(C=0.1, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto', n_jobs=-1,
                                                 penalty='l2', random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini...
                                                     criterion='gini',
                  

In [ ]:
y_pred = voting_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.6714542190305206

Accuracy такая же, как и у случайного леса, добавление других классификаторов её не улучшило. Попробуем BaggingClassifier на случайном лесе с разными значениями n_estimators.

In [ ]:
bagging_clf = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=44, n_jobs=-1,
                                          random_state=42), n_jobs=-1)

In [ ]:
param_grid = {'n_estimators': [5, 10, 15],
              'random_state': [42]}

cv_bagging_clf = GridSearchCV(bagging_clf, param_grid, scoring='accuracy',
                              n_jobs=-1, verbose=2)

In [ ]:
cv_bagging_clf.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  4.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                                               ccp_alpha=0.0,
                                                                               class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=44,
                                                                               max_features='auto',
                                                                               max_leaf_nodes=None,
                                                                               max_samples=None,
                                                                               min_impurity_decrease=0.0,
                                                                               mi

In [ ]:
cv_bagging_clf.best_params_

{'n_estimators': 15, 'random_state': 42}

In [ ]:
bagging_clf = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=44, n_jobs=-1,
                                          random_state=42), n_estimators=15,
                                n_jobs=-1, random_state=42)

In [ ]:
bagging_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=44,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
       

In [ ]:
y_pred = bagging_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.6700578495910632

Бэггинг нескольких случайных лесов ни к чему не привёл – accuracy хуже, чем у одного случайного леса. Попробуем StackingClassifier на всех трёх классификаторах с разными значениями cv и passthrough.

In [ ]:
logreg_clf = LogisticRegression(C=0.1, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)
forest_clf = RandomForestClassifier(max_depth=44, n_jobs=-1, random_state=42)

stacking_clf = StackingClassifier([('logreg', logreg_clf),
                                   ('tree', tree_clf),
                                   ('forest', forest_clf)], n_jobs=-1)

In [ ]:
param_grid = {'cv': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'passthrough': [True, False]}

cv_stacking_clf = GridSearchCV(stacking_clf, param_grid, scoring='accuracy',
                               n_jobs=-1, verbose=2)

In [ ]:
cv_stacking_clf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 32.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=None, error_score=nan,
             estimator=StackingClassifier(cv=None,
                                          estimators=[('logreg',
                                                       LogisticRegression(C=0.1,
                                                                          class_weight=None,
                                                                          dual=False,
                                                                          fit_intercept=True,
                                                                          intercept_scaling=1,
                                                                          l1_ratio=None,
                                                                          max_iter=100,
                                                                          multi_class='auto',
                                                                          n_jobs=-1,
                                        

In [ ]:
cv_stacking_clf.best_params_

{'cv': 6, 'passthrough': True}

In [ ]:
logreg_clf = LogisticRegression(C=0.1, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=8, random_state=42)
forest_clf = RandomForestClassifier(max_depth=44, n_jobs=-1, random_state=42)


stacking_clf = StackingClassifier([('logreg', logreg_clf),
                                   ('tree', tree_clf),
                                   ('forest', forest_clf)], cv=6, n_jobs=-1,
                                  passthrough=True)

In [ ]:
stacking_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


StackingClassifier(cv=6,
                   estimators=[('logreg',
                                LogisticRegression(C=0.1, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=-1, penalty='l2',
                                                   random_state=42,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('tree',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                 

In [ ]:
y_pred = stacking_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.672850588469978

StackingClassifier с подобранными параметрами снова даёт наилучший результат.
#3-class classification


In [ ]:
y_train, y_test = np.floor_divide(y_train, 2), np.floor_divide(y_test, 2)

Посчитаем accuracy наивного классификатора.

In [ ]:
y_pred_naive = np.full(y_test.shape, np.bincount(y_train).argmax())

In [ ]:
y_pred_naive

array([2, 2, 2, ..., 2, 2, 2])

In [ ]:
accuracy_score(y_test, y_pred_naive)

0.7969279872331937

С уменьшением числа классов accuracy наивного классификатора повысилась.

In [ ]:
logreg_clf = LogisticRegression()

param_grid = {'penalty': ['l2', 'none'],
              'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'fit_intercept': [True, False],
              'class_weight': ['balanced', None],
              'random_state': [42],
              'solver': ['newton-sg', 'lbfgs', 'sag', 'saga']}

cv_logreg_clf = GridSearchCV(logreg_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_logreg_clf.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 369 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 652 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 35.4min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', None],
                         'fit_intercept': [True, False],
                         'penalty': ['l2', 'none'], 'random_state': [42],
                         'solver': ['newton-sg', 'lbfgs', 'sag', 'saga']},
             pre_dispatch=

In [ ]:
cv_logreg_clf.best_params_

{'C': 1.5,
 'class_weight': None,
 'fit_intercept': True,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs'}

Изменилось наилучшее значение C для логистической регрессии – для шести классов было 0.1, для трёх стало 1.5.

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)

In [ ]:
logreg_clf.fit(X_train, y_train)

LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=42,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
y_pred = logreg_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8035108717334929

In [ ]:
tree_clf = DecisionTreeClassifier()

param_grid = {'criterion': ['gini','entropy'],
              'splitter': ['best', 'random'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [42]}

cv_tree_clf = GridSearchCV(tree_clf, param_grid, scoring='accuracy', n_jobs=-1,
                           verbose=2)

In [ ]:
cv_tree_clf.fit(X_train, y_train)

Fitting 5 folds for each of 392 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 308 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 860 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1325 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 1912 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1960 out of 1960 | elapsed:  7.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid=...
             param_grid={'class_weight': ['balanced', None],
                         'criterion':

In [ ]:
cv_tree_clf.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'random_state': 42,
 'splitter': 'best'}

Изменилось наилучшее значение max_depth для решающего дерева – для шести классов было 8, для трёх стало 5.

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)

In [ ]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [ ]:
y_pred = tree_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8005186515060841

А сейчас решающее дерево лучше наивного классификатора!

In [ ]:
forest_clf = RandomForestClassifier(n_jobs=-1)

param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'random_state': [42]}

cv_forest_clf = GridSearchCV(forest_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_forest_clf.fit(X_train, y_train)

Fitting 5 folds for each of 392 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 1454 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 1960 out of 1960 | elapsed: 22.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1...
             iid='deprecated', n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
          

При классификации на три класса модели обучаются ещё быстрее.

In [ ]:
cv_forest_clf.best_params_

{'criterion': 'entropy',
 'max_depth': 32,
 'n_estimators': 100,
 'random_state': 42}

Изменились наилучшие значения criterion и max_depth для случайного леса – было gini и 44 для шести классов, стало entropy и 32 для трёх.

In [ ]:
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

In [ ]:
forest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=32, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = forest_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8019150209455416

Случайный лес всё ещё даёт самую высокую accuracy. Попробуем ансамблевые методы для трёх классов.

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

voting_clf = VotingClassifier([('logreg', logreg_clf),
                               ('tree', tree_clf),
                               ('forest', forest_clf)], n_jobs=-1)

In [ ]:
param_grid = {'voting': ['hard', 'soft'],
              'weights': list(product(range(1, 6), repeat=3))}

cv_voting_clf = GridSearchCV(voting_clf, param_grid, scoring='accuracy',
                             n_jobs=-1, verbose=2)

In [ ]:
cv_voting_clf.fit(X_train, y_train)

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 36.8min
[Parallel(n_jobs=-1)]: Done 1009 tasks      | elapsed: 57.2min
[Parallel(n_jobs=-1)]: Done 1250 out of 1250 | elapsed: 70.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=VotingClassifier(estimators=[('logreg',
                                                     LogisticRegression(C=1.5,
                                                                        class_weight=None,
                                                                        dual=False,
                                                                        fit_intercept=True,
                                                                        intercept_scaling=1,
                                                                        l1_ratio=None,
                                                                        max_iter=100,
                                                                        multi_class='auto',
                                                                        n_jobs=-1,
                                                                        penalty='l2',
                         

In [ ]:
cv_voting_clf.best_params_

{'voting': 'hard', 'weights': (1, 1, 2)}

Снизился вес случайного леса в VotingClassifier – наилучшие веса для шести классов были (1, 1, 3), для трёх стали (1, 1, 2).

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

voting_clf = VotingClassifier([('logreg', logreg_clf),
                               ('tree', tree_clf),
                               ('forest', forest_clf)], weights=(1, 1, 2),
                              n_jobs=-1)

In [ ]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('logreg',
                              LogisticRegression(C=1.5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto', n_jobs=-1,
                                                 penalty='l2', random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini...
                                                     criterion='entropy',
               

In [ ]:
y_pred = voting_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8019150209455416

Accuracy VotingClassifier со случайным лесом всё ещё такая же, как и у одного случайного леса. Попробуем BaggingClassifier на случайном лесе с разными значениями n_estimators.

In [ ]:
bagging_clf = BaggingClassifier(
    base_estimator=RandomForestClassifier(criterion='entropy', max_depth=32,
                                          n_jobs=-1, random_state=42),
                                n_jobs=-1)

In [ ]:
param_grid = {'n_estimators': [5, 10, 15],
              'random_state': [42]}

cv_bagging_clf = GridSearchCV(bagging_clf, param_grid, scoring='accuracy',
                              n_jobs=-1, verbose=2)

In [ ]:
cv_bagging_clf.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  3.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                                               ccp_alpha=0.0,
                                                                               class_weight=None,
                                                                               criterion='entropy',
                                                                               max_depth=32,
                                                                               max_features='auto',
                                                                               max_leaf_nodes=None,
                                                                               max_samples=None,
                                                                               min_impurity_decrease=0.0,
                                                                              

In [ ]:
cv_bagging_clf.best_params_

{'n_estimators': 5, 'random_state': 42}

Снизилось наилучшее значение n_estimators для BaggingClassifier на случайном лесе – для шести классов было 15, для трёх стало 5.

In [ ]:
bagging_clf = BaggingClassifier(
    base_estimator=RandomForestClassifier(criterion='entropy', max_depth=32,
                                          n_jobs=-1, random_state=42),
                                n_estimators=5, n_jobs=-1, random_state=42)

In [ ]:
bagging_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=32,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
    

In [ ]:
y_pred = bagging_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8009176142030721

Бэггинг нескольких случайных лесов по-прежнему ухудшает accuracy. Попробуем StackingClassifier на всех трёх классификаторах с разными значениями cv и passthrough.

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

stacking_clf = StackingClassifier([('logreg', logreg_clf),
                                   ('tree', tree_clf),
                                   ('forest', forest_clf)], n_jobs=-1)

In [ ]:
param_grid = {'cv': [2, 3, 4, 5, 6, 7, 8, 9, 10],
              'passthrough': [True, False]}

cv_stacking_clf = GridSearchCV(stacking_clf, param_grid, scoring='accuracy',
                               n_jobs=-1, verbose=2)

In [ ]:
cv_stacking_clf.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 31.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=None, error_score=nan,
             estimator=StackingClassifier(cv=None,
                                          estimators=[('logreg',
                                                       LogisticRegression(C=1.5,
                                                                          class_weight=None,
                                                                          dual=False,
                                                                          fit_intercept=True,
                                                                          intercept_scaling=1,
                                                                          l1_ratio=None,
                                                                          max_iter=100,
                                                                          multi_class='auto',
                                                                          n_jobs=-1,
                                        

In [ ]:
cv_stacking_clf.best_params_

{'cv': 4, 'passthrough': False}

Изменились наилучшие значения cv и passthrough для StackingClassifier – было 6 и True для шести классов, стало 4 и False для трёх.

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

stacking_clf = StackingClassifier([('logreg', logreg_clf),
                                   ('tree', tree_clf),
                                   ('forest', forest_clf)], cv=4, n_jobs=-1)

In [ ]:
stacking_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


StackingClassifier(cv=4,
                   estimators=[('logreg',
                                LogisticRegression(C=1.5, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=-1, penalty='l2',
                                                   random_state=42,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('tree',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                 

In [ ]:
y_pred = stacking_clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.8013165769000599

Лучший результат даёт случайный лес с подобранными параметрами.
#"Great" rating precision
Посчитаем, насколько точно модели предсказывают рецепты с высоким рейтингом (precision класса 2).

In [ ]:
logreg_clf = LogisticRegression(C=1.5, random_state=42, n_jobs=-1)

In [ ]:
logreg_clf.fit(X_train, y_train)

LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=42,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
y_pred = logreg_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.806792759812894

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth=5, random_state=42)

In [ ]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [ ]:
y_pred = tree_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.803030303030303

In [ ]:
forest_clf = RandomForestClassifier(criterion='entropy', max_depth=32,
                                    n_jobs=-1, random_state=42)

In [ ]:
forest_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=32, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = forest_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.8064384678076609

In [ ]:
y_pred = voting_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.8063990218055839

In [ ]:
y_pred = bagging_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.8047938249035141

In [ ]:
y_pred = stacking_clf.predict(X_test)

In [ ]:
precision_score(y_test, y_pred, labels=[2], average=None)[0]

0.8076923076923077

А вот здесь лучший результат даёт StackingClassifier. Будем его использовать в качестве финальной модели.
#Final model
Сохраняем модель.

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(stacking_clf, f)

In [ ]:
uploaded = drive.CreateFile({'title': 'model.pkl'})
uploaded.SetContentFile('model.pkl')
uploaded.Upload()

#New data preparation
##nutrients.csv

In [ ]:
columns = ['Protein', 'Carbohydrate, by difference', 'Fiber, total dietary',
           'Total lipid (fat)', 'Fatty acids, total saturated', 'Cholesterol',
           'Vitamin A, RAE', 'Thiamin', 'Riboflavin', 'Niacin', 'Vitamin B-6',
           'Folate, DFE', 'Choline, total', 'Vitamin B-12',
           'Vitamin C, total ascorbic acid', 'Vitamin D (D2 + D3)',
           'Vitamin E (alpha-tocopherol)', 'Vitamin K (phylloquinone)',
           'Calcium, Ca', 'Phosphorus, P', 'Magnesium, Mg', 'Iron, Fe',
           'Zinc, Zn', 'Copper, Cu', 'Selenium, Se', 'Potassium, K',
           'Sodium, Na']

In [ ]:
daily_values = pd.Series([50, 275, 28, 78, 20, 300, 900, 1.2, 1.3, 16, 1.7, 400,
                          550, 2.4, 90, 20, 15, 120, 1300, 1250, 420, 18, 11,
                          0.9, 55, 4700, 2300], index=columns)

In [ ]:
daily_values

Protein                             50.0
Carbohydrate, by difference        275.0
Fiber, total dietary                28.0
Total lipid (fat)                   78.0
Fatty acids, total saturated        20.0
Cholesterol                        300.0
Vitamin A, RAE                     900.0
Thiamin                              1.2
Riboflavin                           1.3
Niacin                              16.0
Vitamin B-6                          1.7
Folate, DFE                        400.0
Choline, total                     550.0
Vitamin B-12                         2.4
Vitamin C, total ascorbic acid      90.0
Vitamin D (D2 + D3)                 20.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Calcium, Ca                       1300.0
Phosphorus, P                     1250.0
Magnesium, Mg                      420.0
Iron, Fe                            18.0
Zinc, Zn                            11.0
Copper, Cu                           0.9
Selenium, Se    

In [ ]:
data = []

for col in tqdm(df.columns[1:3]):
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?query={col}&dataType=Survey%20(FNDDS)'
    print(url)
    response = requests.get(url, auth=(api_key, ''))
    print(response.json()['foods'][0])
    food = response.json()['foods'][0]['foodNutrients']
    data.append({item['nutrientName']: item['nutrientNumber'] for item in food})
    time.sleep(5)

  0%|          | 0/2 [00:00<?, ?it/s]

https://api.nal.usda.gov/fdc/v1/foods/search?query=amaretto&dataType=Survey%20(FNDDS)
{'fdcId': 789602, 'description': 'Cordial or liqueur', 'additionalDescriptions': 'amaretto; anisette; benedictine; chartreuse; cointreau; creme de menthe; curacao; drambuie; grenadine; kirsch; kummel; sloe gin; triple sec; schnapps; Jagermeister', 'dataType': 'Survey (FNDDS)', 'foodCode': '93201000', 'publishedDate': '2020-04-01', 'foodNutrients': [{'nutrientId': 1003, 'nutrientName': 'Protein', 'nutrientNumber': '203', 'unitName': 'G', 'value': 0.0}, {'nutrientId': 1004, 'nutrientName': 'Total lipid (fat)', 'nutrientNumber': '204', 'unitName': 'G', 'value': 0.3}, {'nutrientId': 1005, 'nutrientName': 'Carbohydrate, by difference', 'nutrientNumber': '205', 'unitName': 'G', 'value': 41.6}, {'nutrientId': 1008, 'nutrientName': 'Energy', 'nutrientNumber': '208', 'unitName': 'KCAL', 'value': 371.0}, {'nutrientId': 1018, 'nutrientName': 'Alcohol, ethyl', 'nutrientNumber': '221', 'unitName': 'G', 'value': 29

 50%|█████     | 1/2 [00:05<00:05,  5.44s/it]

https://api.nal.usda.gov/fdc/v1/foods/search?query=anchovy&dataType=Survey%20(FNDDS)
{'fdcId': 782422, 'description': 'Anchovy, canned', 'dataType': 'Survey (FNDDS)', 'foodCode': '26101180', 'publishedDate': '2020-04-01', 'foodNutrients': [{'nutrientId': 1003, 'nutrientName': 'Protein', 'nutrientNumber': '203', 'unitName': 'G', 'value': 28.89}, {'nutrientId': 1004, 'nutrientName': 'Total lipid (fat)', 'nutrientNumber': '204', 'unitName': 'G', 'value': 9.71}, {'nutrientId': 1005, 'nutrientName': 'Carbohydrate, by difference', 'nutrientNumber': '205', 'unitName': 'G', 'value': 0.0}, {'nutrientId': 1008, 'nutrientName': 'Energy', 'nutrientNumber': '208', 'unitName': 'KCAL', 'value': 210.0}, {'nutrientId': 1018, 'nutrientName': 'Alcohol, ethyl', 'nutrientNumber': '221', 'unitName': 'G', 'value': 0.0}, {'nutrientId': 1051, 'nutrientName': 'Water', 'nutrientNumber': '255', 'unitName': 'G', 'value': 50.3}, {'nutrientId': 1057, 'nutrientName': 'Caffeine', 'nutrientNumber': '262', 'unitName': '

100%|██████████| 2/2 [00:10<00:00,  5.45s/it]


In [ ]:
nutrients = pd.DataFrame(data, index=df.columns[1:], columns=columns,
                         dtype=float).fillna(0.0)

In [ ]:
nutrients.head()

,Protein,"Carbohydrate, by difference","Fiber, total dietary",Total lipid (fat),"Fatty acids, total saturated",Cholesterol,"Vitamin A, RAE",Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, DFE","Choline, total",Vitamin B-12,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Calcium, Ca","Phosphorus, P","Magnesium, Mg","Iron, Fe","Zinc, Zn","Copper, Cu","Selenium, Se","Potassium, K","Sodium, Na"
amaretto,203.0,205.0,291.0,204.0,606.0,601.0,320.0,404.0,405.0,406.0,415.0,435.0,421.0,418.0,401.0,328.0,323.0,430.0,301.0,305.0,304.0,303.0,309.0,312.0,317.0,306.0,307.0
anchovy,203.0,205.0,291.0,204.0,606.0,601.0,320.0,404.0,405.0,406.0,415.0,435.0,421.0,418.0,401.0,328.0,323.0,430.0,301.0,305.0,304.0,303.0,309.0,312.0,317.0,306.0,307.0
apple,203.0,205.0,291.0,204.0,606.0,601.0,320.0,404.0,405.0,406.0,415.0,435.0,421.0,418.0,401.0,328.0,323.0,430.0,301.0,305.0,304.0,303.0,309.0,312.0,317.0,306.0,307.0
apple juice,203.0,205.0,291.0,204.0,606.0,601.0,320.0,404.0,405.0,406.0,415.0,435.0,421.0,418.0,401.0,328.0,323.0,430.0,301.0,305.0,304.0,303.0,309.0,312.0,317.0,306.0,307.0
asparagus,203.0,205.0,291.0,204.0,606.0,601.0,320.0,404.0,405.0,406.0,415.0,435.0,421.0,418.0,401.0,328.0,323.0,430.0,301.0,305.0,304.0,303.0,309.0,312.0,317.0,306.0,307.0


In [ ]:
nutrients = nutrients.divide(daily_values)

In [ ]:
nutrients.head()

,Protein,"Carbohydrate, by difference","Fiber, total dietary",Total lipid (fat),"Fatty acids, total saturated",Cholesterol,"Vitamin A, RAE",Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, DFE","Choline, total",Vitamin B-12,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Calcium, Ca","Phosphorus, P","Magnesium, Mg","Iron, Fe","Zinc, Zn","Copper, Cu","Selenium, Se","Potassium, K","Sodium, Na"
amaretto,4.06,0.745455,10.392857,2.615385,30.3,2.003333,0.355556,336.666667,311.538462,25.375,244.117647,1.0875,0.765455,174.166667,4.455556,16.4,21.533333,3.583333,0.231538,0.244,0.72381,16.833333,28.090909,346.666667,5.763636,0.065106,0.133478
anchovy,4.06,0.745455,10.392857,2.615385,30.3,2.003333,0.355556,336.666667,311.538462,25.375,244.117647,1.0875,0.765455,174.166667,4.455556,16.4,21.533333,3.583333,0.231538,0.244,0.72381,16.833333,28.090909,346.666667,5.763636,0.065106,0.133478
apple,4.06,0.745455,10.392857,2.615385,30.3,2.003333,0.355556,336.666667,311.538462,25.375,244.117647,1.0875,0.765455,174.166667,4.455556,16.4,21.533333,3.583333,0.231538,0.244,0.72381,16.833333,28.090909,346.666667,5.763636,0.065106,0.133478
apple juice,4.06,0.745455,10.392857,2.615385,30.3,2.003333,0.355556,336.666667,311.538462,25.375,244.117647,1.0875,0.765455,174.166667,4.455556,16.4,21.533333,3.583333,0.231538,0.244,0.72381,16.833333,28.090909,346.666667,5.763636,0.065106,0.133478
asparagus,4.06,0.745455,10.392857,2.615385,30.3,2.003333,0.355556,336.666667,311.538462,25.375,244.117647,1.0875,0.765455,174.166667,4.455556,16.4,21.533333,3.583333,0.231538,0.244,0.72381,16.833333,28.090909,346.666667,5.763636,0.065106,0.133478


##recipes.csv